# 2. Data Preprocessing Pipeline


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードの「Data Preprocessing Pipeline（データ前処理パイプライン）」について、ソクラテス式チャットボットプロジェクトの文脈で説明させていただきます。

大きく分けて5つの重要なステップがあります：

### 1. トークナイザーの設定 (2.1)
```python
tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    trust_remote_code=True,
    token=huggingface_token 
)
```
これは、テキストを機械が理解できる数値（トークン）に変換するツールの準備です。例えば「君は幸せについてどう考えるかね？」という文章を、LLMが理解できる数値の配列に変換します。

### 2. データの検証 (2.2)
```python
def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    # ...
```
これは、対話データが正しい形式かチェックします。例えば：
- メッセージが「user」か「model」のものか
- 内容がちゃんとテキストとして保存されているか
などを確認します。

### 3. データセットの準備 (2.3)
```python
def prepare_dataset():
    conversations = []
    # ...
```
このステップでは：
1. 対話データを読み込み
2. ユーザーとモデル（ソクラテス）の会話ペアを作成
3. Gemmaモデル用の形式に変換
4. 長すぎる会話は除外

例えば：
```json
{
    "role": "user",
    "content": "幸せとは何でしょうか？"
},
{
    "role": "model",
    "content": "君にとって幸せとは何かね？"
}
```
このような対話ペアを処理します。

### 4. 特別な処理 (2.4)
```python
def preprocess_function(examples):
    socratic_patterns = [
        "かね", "だろうか", "のかね", "ではないかね",
        # ...
    ]
```
ここでは、ソクラテス式の特徴的な表現（「かね」「だろうか」など）に特別な注目を集めるよう処理します。これにより、モデルがこれらの表現パターンをより効果的に学習できるようになります。

### 5. 最終的なデータセット作成 (2.5)
```python
dataset = prepare_dataset()
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    # ...
)
```
すべての処理を施したデータを、効率的に学習できるよう最終的な形に整えます。

このパイプライン全体の目的は、生の対話データを、ソクラテス式の口調を効果的に学習できる形に整えることです。特に「かね」「だろうか」といった特徴的な表現パターンを強調することで、モデルがソクラテス風の話し方を身につけやすくなるよう工夫されています。

        
</div>
    </details>
</div>


# 2.1 Tokenizer Setup and Initialization

In [ ]:
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    token=huggingface_token 
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

トークナイザーの設定について、より詳しく説明させていただきます。

### トークナイザーとは？
トークナイザーは、人間の言葉をAIが理解できる数値（トークン）に変換する翻訳機のようなものです。

例えば、以下のようなソクラテス式の発言があるとします：
```
「君は幸せについて、どのように考えるかね？」
```

トークナイザーは、この文章を以下のような数値の配列に変換します（実際の数値は例示です）：
```
[2458, 15, 9874, 234, 567, 8901, 45, 789]
```

### コードの詳細説明
```python
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    token=huggingface_token 
)
```

1. `model_name = "google/gemma-2-2b-jpn-it"`
   - 使用するAIモデルを指定しています
   - このモデルは日本語に特化した比較的小さめのモデルです

2. `AutoTokenizer.from_pretrained()`
   - Gemmaモデル専用のトークナイザーを読み込みます
   - このトークナイザーは日本語テキストを適切に処理できるよう特別に調整されています

3. `trust_remote_code=True`
   - Gemmaモデル専用の特別な処理ルールを使用可能にします
   - 例：「かね」「だろうか」などのソクラテス式の表現を適切に処理

4. `token=huggingface_token`
   - Hugging Faceというサービスからモデルを安全にダウンロードするための認証キー

### 具体的な処理例
このトークナイザーは以下のような処理を行います：

1. 文章の分割：
```python
「君は幸せについて、どのように考えるかね？」
↓
['君', 'は', '幸せ', 'について', '、', 'どのように', '考える', 'かね', '？']
```

2. 数値への変換：
```python
'君' → 2458
'は' → 15
'幸せ' → 9874
'について' → 234
...
'かね' → 789
```

3. 特別なトークンの追加：
- 文章の始まりを示す特別な数値（BOS token）
- 文章の終わりを示す特別な数値（EOS token）
- 必要に応じてパディング（埋め合わせ）用の数値

このように、トークナイザーは人間の言葉をAIが処理できる形に変換する重要な役割を果たします。特に今回のプロジェクトでは、ソクラテス式の特徴的な表現（「かね」「だろうか」など）を適切に処理できることが重要です。

        
</div>
    </details>
</div>


# 2.2 Data Validation Functions

In [ ]:
def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True



def validate_dataset(dataset):
    """Validate dataset structure"""
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分は、データの形式が正しいかをチェックする2つの検証関数について説明します。

### 1. validate_message_format関数
```python
def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True
```

この関数は、各メッセージが正しい形式かをチェックします。具体的には：

1. メッセージが辞書型（dict）かチェック
   ```python
   # 正しい形式：
   {
       "role": "user",
       "content": "幸せとは何でしょうか？"
   }
   
   # 間違った形式：
   ["user", "幸せとは何でしょうか？"]  # これは配列なのでエラー
   ```

2. 必要なキー（'role'と'content'）があるかチェック
   ```python
   # 正しい形式：
   {
       "role": "model",
       "content": "君は幸せについてどう考えるかね？"
   }
   
   # 間違った形式：
   {
       "role": "model"  # contentが無いのでエラー
   }
   ```

3. roleが'user'か'model'のどちらかかチェック
   ```python
   # 正しい形式：
   {"role": "user", "content": "..."}
   {"role": "model", "content": "..."}
   
   # 間違った形式：
   {"role": "assistant", "content": "..."}  # roleが不正なのでエラー
   ```

4. contentが文字列かチェック
   ```python
   # 正しい形式：
   {"role": "model", "content": "君はどう思うかね？"}
   
   # 間違った形式：
   {"role": "model", "content": 123}  # 数値なのでエラー
   ```

### 2. validate_dataset関数
```python
def validate_dataset(dataset):
    """Validate dataset structure"""
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset
```

この関数は、データセット全体の構造を確認します：

1. データセットの最初の項目を取り出して内容を表示
   ```python
   # 表示される例：
   Validated first item structure:
   Keys: dict_keys(['input_ids', 'attention_mask'])
   input_ids type: <class 'list'>
   input_ids length: 256
   ```

2. 以下の情報を確認：
   - どんなキーがあるか（input_ids, attention_maskなど）
   - input_ids（トークン化されたテキスト）のデータ型
   - input_idsの長さ（設定した最大長に収まっているか）

これらの検証は、学習データが正しい形式でモデルに渡されることを保証する重要な役割を果たします。データの形式が間違っていると、学習が正しく行われない可能性があるためです。

        
</div>
    </details>
</div>


# 2.3 Data Set Preparation Function

In [ ]:
# 2.3 Data Set Preparation Function
def prepare_dataset():
    conversations = []
    
    try:
        with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
            
        for dialogue in dialogue_data:
            messages = dialogue.get('messages', [])
            
            # Validate message format
            if not all(validate_message_format(msg) for msg in messages):
                logging.warning(f"Skipped dialogue due to invalid message format")
                continue
                
            # Construct conversation in user->model order
            current_conversation = []
            valid_sequence = True
            
            for i in range(0, len(messages)-1, 2):
                if (i+1 < len(messages) and 
                    messages[i]['role'] == 'user' and 
                    messages[i+1]['role'] == 'model'):
                    current_conversation.extend([messages[i], messages[i+1]])
                else:
                    valid_sequence = False
                    break
            
            # Add only valid conversations
            if valid_sequence and current_conversation:
                # Apply Gemma chat template
                formatted_text = tokenizer.apply_chat_template(
                    current_conversation,
                    tokenize=False,
                    add_generation_prompt=True
                )
                
                # Check token count
                tokens = tokenizer.encode(formatted_text)
                if len(tokens) <= MAX_SEQUENCE_LENGTH:
                    conversations.append({"text": formatted_text})
                else:
                    logging.warning(f"Skipped conversation due to length: {len(tokens)} tokens")
            
    except Exception as e:
        logging.error(f"Error processing dialogue file: {str(e)}")
        raise
    
    if not conversations:
        raise ValueError("No valid conversations found in the dialogue file")
        
    logging.info(f"Processed {len(conversations)} valid conversations")
    return Dataset.from_list(conversations)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



`prepare_dataset`関数について、ソクラテス式チャットボットプロジェクトの文脈で説明します。

この関数は、対話データを学習用に整形する重要な処理を行います。主に以下の5つのステップがあります：

### 1. データの読み込み
```python
with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
    dialogue_data = json.load(f)
```
JSONファイルから対話データを読み込みます。例えば：
```json
[
  {
    "messages": [
      {"role": "user", "content": "幸せとは何でしょうか？"},
      {"role": "model", "content": "君は幸せをどのように定義するかね？"}
    ]
  },
  // 他の対話データ...
]
```

### 2. メッセージ形式の検証
```python
if not all(validate_message_format(msg) for msg in messages):
    logging.warning(f"Skipped dialogue due to invalid message format")
    continue
```
各メッセージが正しい形式か確認します。不正な形式の例：
```json
{
    "speaker": "user",  // "role"ではないのでエラー
    "text": "..."      // "content"ではないのでエラー
}
```

### 3. 対話順序の構築と検証
```python
for i in range(0, len(messages)-1, 2):
    if (i+1 < len(messages) and 
        messages[i]['role'] == 'user' and 
        messages[i+1]['role'] == 'model'):
        current_conversation.extend([messages[i], messages[i+1]])
```
必ずuser→modelの順序になっているか確認します：
```python
# 正しい順序：
[
    {"role": "user", "content": "幸せとは何でしょうか？"},
    {"role": "model", "content": "君は幸せをどのように定義するかね？"}
]

# 誤った順序（スキップされる）：
[
    {"role": "model", "content": "..."},  # modelが先
    {"role": "user", "content": "..."}
]
```

### 4. Gemmaモデル用の形式に変換
```python
formatted_text = tokenizer.apply_chat_template(
    current_conversation,
    tokenize=False,
    add_generation_prompt=True
)
```
対話をGemmaモデルが理解できる特別な形式に変換します。例：
```python
# 変換前：
[{"role": "user", "content": "幸せとは？"}, 
 {"role": "model", "content": "君はどう考えるかね？"}]

# 変換後：
"<start_of_turn>user\n幸せとは？<end_of_turn>
<start_of_turn>model\n君はどう考えるかね？<end_of_turn>"
```

### 5. 長さの確認と最終データセット作成
```python
tokens = tokenizer.encode(formatted_text)
if len(tokens) <= MAX_SEQUENCE_LENGTH:
    conversations.append({"text": formatted_text})
```
対話が長すぎないか確認し（256トークン以内）、問題なければ最終的なデータセットに追加します。

最後に、有効な対話が1つも無い場合はエラーを発生させ、あれば処理済みの対話データをデータセット形式で返します：
```python
return Dataset.from_list(conversations)
```

この関数は、生の対話データを、ソクラテス式チャットボットの学習に適した形式に変換する重要な役割を果たしています。特に、user→modelの順序を保証することで、ソクラテスのような応答パターンを学習しやすくしています。

        
</div>
    </details>
</div>


# 2.4 Data Processing Pipeline Function

In [ ]:
# 2.4 Data Processing Pipeline Function
def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_TOKENIZE_LENGTH,      # Use global setting
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result

def preprocess_function(examples):
    # Focus on Socratic tone and inquiry patterns
    socratic_patterns = [
        # Question patterns
        "かね", "だろうか", "のかね", "ではないかね",
        # Question introduction
        "では", "について",
        # Second person (characteristic of mature tone)
        "君は", "君が", "君の"
    ]
    
    # Get tokenized text
    texts = tokenizer.batch_decode(examples['input_ids'])
    new_attention_masks = []
    
    for text, mask in zip(texts, examples['attention_mask']):
        if not isinstance(mask, list):
            mask = mask.tolist()

        new_mask = mask.copy() 
        
        # Split text
        sentences = text.split('。')
        current_pos = 0
        
        for sentence in sentences:
            if not sentence.strip():
                continue
            
            # Detect and highlight Socratic patterns
            for pattern in socratic_patterns:
                if pattern in sentence:
                    # Identify pattern position
                    pattern_tokens = tokenizer.encode(pattern, add_special_tokens=False)
                    pattern_len = len(pattern_tokens)
                    
                    # Highlight tokens containing the pattern and its surroundings
                    pattern_start = current_pos + len(tokenizer.encode(sentence, add_special_tokens=False)) - pattern_len
                    for i in range(max(0, pattern_start - 2), min(len(mask), pattern_start + pattern_len + 2)):
                        new_mask[i] = 1.0  # Max attention to pattern part
            
            # Update position for each sentence segment
            current_pos += len(tokenizer.encode(sentence + '。', add_special_tokens=False))
        
        # Special token masks are set to 1.0
        if tokenizer.bos_token_id is not None:
            new_mask[0] = 1.0  # BOS token
        if tokenizer.eos_token_id is not None:
            new_mask[-1] = 1.0  # EOS token
            
        new_attention_masks.append(new_mask)

    examples['attention_mask'] = new_attention_masks
    return examples



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分は、データの前処理を行う2つの重要な関数について説明します。

### 1. tokenize_function
```python
def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_TOKENIZE_LENGTH,
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result
```

この関数は、テキストをトークン化（数値化）します。

例えば：
```python
# 入力テキスト
text = "君は幸せについて、どのように考えるかね？"

# トークン化の結果
result = {
    'input_ids': [1, 245, 15, 987, 234, 567, 890, 45, 789, 2],  # 数値化されたテキスト
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # どの部分を注目するか
}
```

パラメータの説明：
- `truncation=True`: 長すぎるテキストを切り詰める
- `max_length=MAX_TOKENIZE_LENGTH`: 最大256トークンまで
- `padding='max_length'`: 短いテキストを最大長まで埋める
- `add_special_tokens=True`: 文章の開始・終了を示す特別なトークンを追加

### 2. preprocess_function
```python
def preprocess_function(examples):
    socratic_patterns = [
        # Question patterns
        "かね", "だろうか", "のかね", "ではないかね",
        # Question introduction
        "では", "について",
        # Second person
        "君は", "君が", "君の"
    ]
    # ...
```

この関数は、ソクラテス式の特徴的な表現に特別な注目を集めるよう処理します。

#### 処理の流れ：

1. ソクラテス式パターンの定義
```python
# 特に注目する表現パターン
"かね"        # 例：「どう思うかね？」
"だろうか"    # 例：「それは本当だろうか？」
"君は/が/の"  # 例：「君はどう考える？」
```

2. パターンの検出と強調
```python
# 例：「君は幸せについて、どのように考えるかね？」という文で
# 「君は」と「かね」を検出して強調

# attention_maskの例（1.0が強調部分）
[1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 1.0, 1.0]
#  ↑    ↑                         ↑    ↑
# 「君」「は」        ...        「か」「ね」
```

3. 特別トークンの処理
```python
# 文章の開始(BOS)と終了(EOS)トークンも強調
if tokenizer.bos_token_id is not None:
    new_mask[0] = 1.0  # 開始トークン
if tokenizer.eos_token_id is not None:
    new_mask[-1] = 1.0  # 終了トークン
```

この処理により、モデルは：
- 「かね」「だろうか」などの質問パターン
- 「君は」などの二人称表現
- 「では」「について」などの導入表現

に特に注目して学習を行います。これにより、ソクラテス式の口調をより効果的に学習することができます。

例えば、モデルは以下のような応答パターンを学習しやすくなります：
```
User: 幸せとは何でしょうか？
Model: 君は幸せについて、どのように考えるかね？
       ↑    ↑         ↑           ↑
       特に注目するパターン
```

        
</div>
    </details>
</div>



# 2.5 Data Set Preparation

In [ ]:
# 2.5 Data Set Preparation
# Prepare base dataset
dataset = prepare_dataset()
logging.info(f"Total dataset size: {len(dataset)}")

# Validate dataset structure
print("Dataset structure:")
print(dataset[0])  # Display first element
print("\nDataset features:")
print(dataset.features)

# Optimize dataset batch processing
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)

# Optimize dataset processing
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,  
    num_proc=4,     
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)

# Apply preprocessing
tokenized_dataset = tokenized_dataset.map(
    preprocess_function,
    batched=True,
    desc="Applying attention masking"
)

# Final dataset validation
tokenized_dataset = validate_dataset(tokenized_dataset)



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分は、データセットの最終的な準備段階について説明します。順を追って解説していきます。

### 1. 基本データセットの準備と確認
```python
dataset = prepare_dataset()
logging.info(f"Total dataset size: {len(dataset)}")

# データセット構造の確認
print("Dataset structure:")
print(dataset[0])  # 最初の要素を表示
```

例えば以下のような出力が得られます：
```python
Total dataset size: 5000  # 総対話数

Dataset structure:
{
    'text': '<start_of_turn>user\n幸せとは何でしょうか？<end_of_turn>\n<start_of_turn>model\n君は幸せをどのように定義するかね？<end_of_turn>'
}
```

### 2. データセットのシャッフルと最適化
```python
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)
```

- データをランダムに並び替えて、学習の偏りを防ぎます
- `seed=42`で、毎回同じシャッフル結果になるようにしています
- 例えば：
  ```python
  # シャッフル前
  1. 幸せについての対話
  2. 正義についての対話
  3. 知識についての対話
  
  # シャッフル後
  1. 知識についての対話
  2. 幸せについての対話
  3. 正義についての対話
  ```

### 3. トークン化の実行
```python
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,  # 32個ずつ処理
    num_proc=4,     # 4つのプロセスで並列処理
    load_from_cache_file=True,  # キャッシュを使用
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)
```

例えば：
```python
# 変換前
"君は幸せについて、どのように考えるかね？"

# 変換後
{
    'input_ids': [1, 245, 15, 987, ...],  # 数値化されたテキスト
    'attention_mask': [1, 1, 1, 1, ...]   # 注目する部分
}
```

### 4. ソクラテス式パターンの処理適用
```python
tokenized_dataset = tokenized_dataset.map(
    preprocess_function,
    batched=True,
    desc="Applying attention masking"
)
```

この処理で、ソクラテス式の特徴的な表現により注目するように設定します：
```python
# 例：「君は幸せについて、どのように考えるかね？」

# attention_maskの値
[1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 1.0, 1.0]
#  ↑    ↑                         ↑    ↑
# 「君」「は」        ...        「か」「ね」
# （1.0は特に注目する部分）
```

### 5. 最終確認
```python
tokenized_dataset = validate_dataset(tokenized_dataset)
```

最終的なデータセットの構造を確認します：
```python
Validated first item structure:
Keys: dict_keys(['input_ids', 'attention_mask'])
input_ids type: <class 'list'>
input_ids length: 256
```

この一連の処理により、生の対話データが：
1. 適切にシャッフルされ
2. 数値化され
3. ソクラテス式の特徴が強調され
4. 効率的に処理できる形に整理された

最終的なデータセットが完成します。これにより、モデルがソクラテス式の対話パターンを効果的に学習できる準備が整います。

        
</div>
    </details>
</div>
